In [2]:
import pandas as pd
import numpy as np

In [10]:
original_data = pd.read_csv("Original Data/original_data.csv", low_memory=False)

In [11]:
# prune all variables with mv > 50%
original_data.describe()

,login,grp,xsect,dem02,dem03,dem04,dem05,dem05_32,dem05_16,dem05_8,...,nvalid_ph,ph_t,nvalid_mh,mh_t,sgrq_sx_r,sgrq_act_r,sgrq_imp_r,sgrq_tot_r,Unnamed: 949,Unnamed: 950
count,2796.000000,2147.000000,2793.000000,2796.000000,2796.000000,2777.000000,2750.000000,2729.000000,2750.0,2750.000000,...,341.000000,336.000000,336.000000,336.000000,3.340000e+02,3.320000e+02,3.310000e+02,3.310000e+02,9.000000e+00,1.000000e+00
mean,57436.652361,1.674429,1.625492,63.317597,1.442418,0.013684,1.622182,0.002565,0.0,0.029091,...,5.187683,350.059524,12.050595,392.160714,1.247146e+16,9.502412e+15,1.408852e+16,1.243517e+16,1.404604e+16,5.985361e+15
std,24407.716248,0.468697,0.484082,9.566788,0.496762,0.116196,2.303022,0.050591,0.0,0.168092,...,20.420289,133.354010,55.637423,164.689118,1.554408e+16,1.423946e+16,1.580091e+16,1.491821e+16,2.121146e+16,NaN
min,10341.000000,1.000000,1.000000,41.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,...,3.000000,4.000000,2.000000,4.000000,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e+01,5.312472e+14,5.985361e+15
25%,38692.000000,1.000000,1.000000,55.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,...,4.000000,308.000000,4.000000,347.250000,4.865522e+15,8.451879e+14,4.448484e+15,4.784877e+15,7.677846e+14,5.985361e+15
50%,55474.000000,2.000000,2.000000,62.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,...,4.000000,369.000000,4.000000,426.000000,6.921509e+15,6.630552e+15,6.676740e+15,6.652819e+15,4.704467e+15,5.985361e+15
75%,78680.000000,2.000000,2.000000,71.000000,2.000000,0.000000,2.000000,0.000000,0.0,0.000000,...,4.000000,421.000000,4.000000,502.500000,9.280755e+15,8.526999e+15,1.889807e+16,8.677027e+15,7.276533e+15,5.985361e+15
max,97908.000000,2.000000,2.000000,88.000000,2.000000,1.000000,32.000000,1.000000,0.0,1.000000,...,381.000000,677.000000,482.000000,676.000000,6.336604e+16,6.176495e+16,6.352347e+16,7.156961e+16,5.141577e+16,5.985361e+15


In [43]:
from sklearn.impute import SimpleImputer

# Perform per-patient missing value imputation
from pandas import DataFrame
from pandas import concat
# first row are the variable names
# second row are the variable types
data = pd.read_excel("working_sheet.xlsx", sheet_name="variable_types", header=0)

var_names = data.columns
# get the second row
var_types = data.iloc[0, :]
# print(data.shape)
# drop the second row
data = data.drop(0, axis=0)
# print(data.shape)

numeric_vars = var_names[var_types == 'Numeric'].tolist()
categorical_vars = var_names[var_types == 'Categorical'].tolist()


# iterate through all unique values in the login column. Skip the second header row
for login in data.loc[:,"login"].unique():
    # get a selection of all rows with that login value
    login_selection = data[data["login"] == login]
    print(login_selection)
    
    # use median strategy to impute numeric vars
    imp = SimpleImputer(strategy='constant', copy=True, missing_values=np.nan, verbose=1)
    tmp_nr = DataFrame(imp.fit_transform(login_selection[numeric_vars]), columns=numeric_vars)
    
    imp = SimpleImputer(strategy='most_frequent', copy=True, missing_values=np.nan, verbose=1)
    tmp_cat = DataFrame(imp.fit_transform(login_selection[categorical_vars]), columns=categorical_vars)
    
    df = concat([tmp_nr, tmp_cat], axis=1)
    # replace the original data with the imputed data
    data.loc[login_selection.index, :] = df
# write to a new file
data.to_csv("per_patient_imputation.csv", sep=";")


    login dem02 dem03 dem04 dem05 dem05_32 dem05_16 dem05_8 dem05_4 dem05_2  \
1   10341    55     1     0     1        0        0       0       0       0   
2   10341    55     1     0     1        0        0       0       0       0   
3   10341    55     1     0     1        0        0       0       0       0   
4   10341    55     1     0     1        0        0       0       0       0   
5   10341    55     1     0     1        0        0       0       0       0   
6   10341    55     1     0     1        0        0       0       0       0   
7   10341    55     1     0     1        0        0       0       0       0   
8   10341    55     1     0     1        0        0       0       0       0   
9   10341    55     1     0     1        0        0       0       0       0   
10  10341    55     1     0     1        0        0       0       0       0   
11  10341    55     1     0     1        0        0       0       0       0   
12  10341    55     1     0     1        0        0 

c:\Users\Daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
c:\Users\Daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
c:\Users\Daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_base.py:580: UserWarning: Skipping features without any observed values: ['bclinpt25' 'bclinpt27' 'bclinra08' 'fclinra03' 'bclinra09' 'bclinra10'
 'fclinra09' 'fclinra12' 'fclinra15' 'An1_Bank' 'An2_Bank' 'An4_Bank'
 'An5_Bank' 'An7_Bank' 'An8_Bank' 'An12_Bank' 'An14_Bank' 'An15_Bank'
 'An16_Bank'

ValueError: Shape of passed values is (18, 411), indices imply (18, 800)

In [ ]:
import pandas_profiling
data = pd.read_csv("per_patient_imputation.csv", sep=";")

# count all cells that contain ''
# print(data[data == ''].count().sum() / data.size)



print(data.isna().sum().sum() / data.size)

# report = pandas_profiling.ProfileReport(data)
# report.to_file("report_cleaning_1.html")


C:\Users\Daniel\AppData\Local\Temp\ipykernel_4280\1429530462.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,231,233,235,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270

0.761733279390826
